# SocraticFlanT5 - Evaluation | DL2 Project, May 2023
---

This notebook evaluates the generated captions based on the MS COCO ground-truth captions. We will evaluate the folowing two approaches: 
1. <span style="color:#006400">**Baseline**</span>: a Socratic model based on the work by [Zeng et al. (2022)](https://socraticmodels.github.io/) where GPT-3 is replaced by [FLAN-T5-xl](https://huggingface.co/docs/transformers/model_doc/flan-t5). 

2. <span style="color:#006400">**Improved prompting**</span>: an improved baseline model where the template prompt filled by CLIP is processed before passing to FLAN-T5-xl.

There are two approaches to this evaluation: rule-based and embedding-based.

---
For the **rule-based approach**, the following metrics will be used, based on [this](https://github.com/salaniz/pycocoevalcap) repository:

* *BLEU-4*: BLEU (Bilingual Evaluation Understudy) is a metric that measures the similarity between the generated captions and the ground truth captions based on n-gram matching. The BLEU-4 score measures the precision of the generated captions up to four-grams compared to the ground truth captions.

* *METEOR*: METEOR (Metric for Evaluation of Translation with Explicit ORdering) is another metric that measures the similarity between the generated captions and the ground truth captions. It also takes into account word order and synonymy by using a set of reference summaries to compute a harmonic mean of precision and recall.

* *CIDEr*: CIDEr (Consensus-based Image Description Evaluation) is a metric that measures the consensus between the generated captions and the ground truth captions. It computes the similarity between the generated captions and the reference captions based on their TF-IDF weights, which helps capture important words in the captions.

* *SPICE*: SPICE (Semantic Propositional Image Caption Evaluation) is a metric that measures the semantic similarity between the generated captions and the ground truth captions. It analyzes the similarity between the semantic propositions present in the generated captions and those in the reference captions, taking into account the structure and meaning of the propositions.

* *ROUGE-L*: ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a metric that measures the similarity between the generated captions and the ground truth captions based on overlapping sequences of words. ROUGE-L measures the longest common subsequence (LCS) between the generated captions and the reference captions, taking into account sentence-level structure and word order.

---

For the **embedding-based** approach (based on CLIP embeddings), we calculate the cosine similarities between each image embedding and embeddings of the ground truth captions and then we calculate the cosine similarities between each image embedding and embeddings of the captions generated with FLAN-T5-xl.

---

<span style="color:#8B0000">**Important**</span>: we assume that you have the generated captions accessible from the current directory via `cache/baseline_outputs.csv` or `cache/improved_outputs.csv` or both. If that is not the case, please run (either or both of) the following notebooks:
* `SocraticFlanT5 - Caption Generation_baseline.ipynb`
* `SocraticFlanT5 - Caption Generation_improved.ipynb`

Moreover, we assume you have pre-computed the image embeddings and have them stored and accessible from the current directory via `cache/embed_imgs.pickle`. If that is not the case, please run (either or both of) the following notebook:
* `SocraticFlanT5 - Image Embedding Generation.ipynb`

### Set-up

#### Loading the required packages

In [1]:
# Package loading
import os
import json
import pickle
import pandas as pd

# Local imports
from image_captioning import ClipManager
from eval import SocraticEvalCap
from utils import get_device

### Evaluate the generated captions against the ground truth

#### Load the ground truth annotations

In [2]:
annotation_file = 'annotations/annotations/captions_val2017.json'

with open(annotation_file, 'r') as f:
    lines = json.load(f)['annotations']
gts = {}
for item in lines:
    if item['image_id'] not in gts:
        gts[item['image_id']] = []
    gts[item['image_id']].append({'image_id': item['image_id'], 'caption': item['caption']})

#### Compute the embeddings for the gt captions

In [3]:
if not os.path.exists('cache/embed_capt_gt.pickle'):
    # Set the device to use
    device = get_device()

    # Instantiate the clip manager
    clip_manager = ClipManager(device)
    
    embed_capt_gt = {}
    for img_id, list_of_capt_dict in gts.items():
        list_of_captions = [capt_dict['caption'] for capt_dict in list_of_capt_dict]

        # Dims of img_feats_gt: 5 x 768
        img_feats_gt = clip_manager.get_text_feats(list_of_captions)

        embed_capt_gt[img_id] = img_feats_gt

    with open('cache/embed_capt_gt.pickle', 'wb') as handle:
        pickle.dump(embed_capt_gt, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Evaluation

In [4]:
approaches = ['baseline', 'improved']
# approaches = ['baseline']


eval_cap = {
    'rulebased': {},
    'cossim': {}
}

for approach in approaches:
    # Load the generated captions
    res_raw = pd.read_csv(f'{approach}_outputs.csv')

    
    evaluator = SocraticEvalCap(gts, res_raw)

    # Rule-based metrics
    evaluator.evaluate_rulebased()
    eval_rulebased = {}

    for metric, score in evaluator.eval.items():
        print(f'{metric}: {score:.3f}')
        eval_rulebased[metric] = round(score, 5)
    eval_cap['rulebased'][approach] = eval_rulebased

    # Embedding-based metric
    evaluator.evaluate_cossim()
    for source_caption, sim in evaluator.sims.items():
        print(f'{source_caption}: avg = {sim[0]:.3f}, std = {sim[1]:.3f}')
    eval_cap['cossim'][approach] = evaluator.sims

tokenization...


PTBTokenizer tokenized 125 tokens at 1826.59 tokens per second.
PTBTokenizer tokenized 58 tokens at 869.81 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 46, 'reflen': 25, 'guess': [46, 44, 42, 40], 'correct': [6, 1, 0, 0]}
ratio: 1.8399999999264
Bleu_1: 0.130
Bleu_2: 0.054
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.058
computing Rouge score...
ROUGE_L: 0.159
computing CIDEr score...
CIDEr: 0.028
computing SPICE score...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 2.068 s
SPICE: 0.030
Bleu_1: 0.130
Bleu_2: 0.054
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.058
ROUGE_L: 0.159
CIDEr: 0.028
SPICE: 0.030
gts: avg = 0.241, std = 0.027
res: avg = 0.188, std = 0.005


### Save the outputs

In [5]:
with open('eval_cap.pickle', 'wb') as handle:
    pickle.dump(eval_cap, handle, protocol=pickle.HIGHEST_PROTOCOL)